In [2]:
import requests
from psycopg2.extras import execute_values
from datetime import datetime, timezone
from ipyleaflet import Map, LayerGroup, Marker, DivIcon, FullScreenControl, basemaps, TileLayer
from ipywidgets import Layout
import psycopg2

plane_svg = """
<div style="transform: rotate({heading}deg); transform-origin: center;">
    <svg width="30px" height="30px" viewBox="0 0 512 512" xmlns="http://www.w3.org/2000/svg">
        <path fill="{color}" d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z"/>
    </svg>
</div>
"""

plane_svg = """
<svg width="30px" height="30px" viewBox="0 0 512 512" version="1.1"
     xmlns="http://www.w3.org/2000/svg"
     style="transform: rotate({heading}deg); background-color: transparent;">
    <path fill="{color}"
          d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z"
          stroke="white" stroke-width="10" stroke-linejoin="round"/>
</svg>
"""

In [3]:
def initialize_database():
    conn = psycopg2.connect(
        host="localhost",
        port="5432",
        database="flight_data",
        user="admin",
        password="admin"
    )
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    print("Initializing Database...")
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS observations (
        time        TIMESTAMPTZ NOT NULL,
        icao24      VARCHAR(10) NOT NULL,
        callsign    VARCHAR(20),
        location    GEOMETRY(POINT, 4326),
        altitude    DOUBLE PRECISION,
        velocity    DOUBLE PRECISION,
        heading     DOUBLE PRECISION,
        on_ground   BOOLEAN
    );
    """)
    try:
        cur.execute("SELECT create_hypertable('observations', 'time');")
        print("Hypertable created successfully.")
    except psycopg2.errors.DuplicateObject:
        print("Hypertable already exists, skipping.")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_icao_time ON observations (icao24, time DESC);")

    cur.close()
    conn.close()
    print("Database is ready!")

In [4]:
def check_row_count():
    conn = psycopg2.connect(host="localhost", database="flight_data", user="admin", password="admin")
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM observations;")
    count = cur.fetchone()[0]
    print(f"Current rows in DB: {count}")
    cur.close()
    conn.close()

In [5]:
def save_to_db(planes):
    # Connection details from your docker-compose
    conn = psycopg2.connect(
        host="localhost",
        port="5432",
        database="flight_data",
        user="admin",
        password="admin"
    )
    cur = conn.cursor()
    # data_to_insert = [
    #     (datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'], p['lon'], p['lat'],
    #      p['altitude'], p['velocity'], p['heading'], p['on_ground'])
    #     for p in planes
    # ]
    #
    # query = """
    # INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
    # VALUES %s
    # ON CONFLICT DO NOTHING;
    # """

    execute_values(cur, """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s""",
        [(datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'], f"SRID=4326;POINT({p['lon']} {p['lat']})",
          p['altitude'], p['velocity'], p['heading'], p['on_ground']) for p in planes])

    conn.commit()
    cur.close()
    conn.close()
    print(f"Successfully saved {len(planes)} rows to TimescaleDB.")

In [6]:
def get_live_planes(username=None, password=None):
    # API Endpoint for all states
    url = "https://opensky-network.org/api/states/all"

    # Optional: Use auth to get higher rate limits (4000/day vs 100/day)
    auth = (username, password) if username and password else None

    try:
        response = requests.get(url, auth=auth, timeout=10)
        response.raise_for_status()
        data = response.json()

        states = data.get('states', [])
        timestamp = data.get('time')

        processed_planes = []

        for s in states:
            # We filter for planes that have valid lat/lon
            if s[5] is not None and s[6] is not None:
                processed_planes.append({
                    "time": timestamp,
                    "icao24": s[0],
                    "callsign": s[1].strip() if s[1] else "NONE",
                    "lon": s[5],
                    "lat": s[6],
                    "altitude": s[7], # Barometric altitude
                    "velocity": s[9],
                    "heading": s[10] or 0, # Default to 0 if null
                    "on_ground": s[8]
                })

        return processed_planes

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

In [8]:
planes = get_live_planes()
print(f"Found {len(planes)} planes with GPS data.")
check_row_count()

Found 9767 planes with GPS data.
Current rows in DB: 49597


In [13]:
# --- 1. Map Initialization (The "No-Repeat" Constraints) ---
# Center on London [51.5074, -0.1278]
m = Map(
    center=(51.5, -0.1),
    zoom=7,
    scroll_wheel_zoom=True,
    world_copy_jump=False,
    basemap=basemaps.OpenStreetMap.Mapnik,
    layout=Layout(width='100%', height='600px')
)
display(m)

# Set hard limits so you can't pan into the "gray void" past the North/South poles
m.max_bounds = [(-90, -180), (90, 180)]

# Add a layer to hold planes
plane_layer = LayerGroup()
m.add_layer(plane_layer)
m.add_control(FullScreenControl())


# --- 2. Spatial Query Function ---
def get_visible_planes(south, west, north, east):
    """Queries TimescaleDB for planes strictly within the current view."""
    try:
        conn = psycopg2.connect(
            host="localhost", port="5432",
            database="flight_data", user="admin", password="admin"
        )
        cur = conn.cursor()

        query = """
        SELECT DISTINCT ON (icao24)
               icao24, callsign, ST_Y(location::geometry) as lat, ST_X(location::geometry) as lon,
               heading, on_ground
        FROM observations
        WHERE location && ST_MakeEnvelope(%s, %s, %s, %s, 4326);
        """

        cur.execute(query, (west, south, east, north))
        data = cur.fetchall()
        cur.close()
        conn.close()
        return data
    except Exception as e:
        print(f"DB Error: {e}")
        return []

# --- 3. The Update Logic ---
def update_view(change=None):
    """Triggered whenever the user moves or zooms."""
    bounds = m.bounds
    if not bounds or len(bounds) < 2:
        # Fallback for initial load
        south, west, north, east = 49.0, -2.0, 53.0, 2.0
    else:
        (south, west), (north, east) = bounds

    # print to logger bounds
    with open('logger', 'w') as f:
        f.write(f"bounds: {bounds}\n")

    # Fetch and filter
    planes = get_visible_planes(south, west, north, east)
    print(f"found {len(planes)} planes with GPS data.")

    # Batch render
    new_markers = []
    for p in planes[:300]:
        icao, callsign, lat, lon, heading, on_ground = p

        # Plane icon with rotation
        color = "#28a745" if on_ground else "#007bff"
        icon_html = plane_svg.format(heading=heading or 0, color=color)
        icon = DivIcon(html=icon_html, bg_pos=[0, 0], icon_size=[30, 30])

        new_markers.append(Marker(location=(lat, lon), icon=icon, title=callsign, draggable=False))

    # Update the map in one shot
    plane_layer.clear_layers()
    plane_layer.layers = tuple(new_markers)

# --- 4. Activation ---
m.observe(update_view, names='bounds')

Map(center=[51.5, -0.1], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…